<a href="https://colab.research.google.com/github/perlitz/Notes/blob/master/MNIST_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self, activation):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.activation = activation 

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def get_mnist_loaders(train_batch_size, test_batch_size):
    """Get MNIST data loaders"""
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True)

    return train_loader, test_loader

In [0]:
def train(log_interval, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [29]:
def main():

    cfg = {'train_batch_size' : 64,
           'test_batch_size' : 1000,
           'n_epochs' : 1,
           'lr' : 0.01,
           'momentum': 0.9,
           'seed' : 0,
           'log_interval' : 100,
           'save_model' : False,
           'optimizer': optim.SGD,
           'activation': F.relu}

    torch.manual_seed(cfg['seed'])
    train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])

    model = Net(cfg['activation'])
    optimizer = cfg['optimizer'](model.parameters(), lr=cfg['lr'])
    for epoch in range(1, cfg['n_epochs'] + 1):
        train(cfg['log_interval'], model, train_loader, optimizer, epoch)
        test(model, test_loader)

    if cfg['save_model']:
        torch.save(model.state_dict(), "mnist_cnn.pt")

if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318375
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.227785
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.119398
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.069649
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.755454
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.550546
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.369271
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.071817
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.085687
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.819594
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.942243
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.571486
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.557941
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.022847
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.805649
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.664481
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.561431
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.452135
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.618941
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.540298
Train Epoch: 1 [12800/60000 (

KeyboardInterrupt: ignored